Examples taken from https://towardsdatascience.com/an-in-depth-tutorial-to-python-decorators-that-you-can-actually-use-1e34d3d2d305

## Functions Are Objects

In [1]:
func_dict = {
    'lower': str.lower,
    'print': print,
    'range': range,
    'startswith': str.startswith
    }

In [2]:
func_dict['print'](func_dict['lower']('PYTHON'))

python


## Scope

Python provides us with a keyword that lets us specify that we are referring to names in the global scope:

In [4]:
a = 24

# Before
def foo():
    a = 'some text'  # still local
    print(a)

foo()
print(a)

some text
24


In [5]:
# Using `global` keyword
# After
def foo():
    global a
    a = 'some text'  # now accessed the `a` from global
    print(a)
foo()
print(a)

some text
some text


Between global and local, there is one level: nonlocal scope comes into play when we have nested functions:

In [6]:
def outer():
     # Create a dummy variable
     my_var = 'Python'
     
     def inner():
         # Try to change the value of the dummy
         my_var = 'Data Science'
         print(my_var)
     # Call the inner function which tries to modify `my_var`
     inner()
     # Check if successful
     print(my_var)
 
outer()

Data Science
Python


We cannot use global keyword since my_var is not in the global scope.

For such cases, you can use nonlocal keyword that gives access to all the names in the scope of the outer function (nonlocal) but not the global:

In [7]:
def outer():
     # Create a dummy variable
     my_var = 'Python'
     
     def inner():
         # Try to change the value of the dummy with nonlocal
         nonlocal my_var
         my_var = 'Data Science'
         print(my_var)
     # Call the inner function which tries to modify `my_var`
     inner()
     # Check if successful
     print(my_var)
 
outer()

Data Science
Data Science


There can be four levels of scope in a single script/program:

- Built-in: all the package names installed with Python, pip and conda
- Global: general scope, all names that have no indentation in the script
- Local: contains local variables in code blocks such as functions, loops, list comprehensions, etc.
- Nonlocal: an extra level of scope between global and local in the case of nested functions

## Closures

In [8]:
def foo():
    x = 42
    
    def bar():
        print(x)
    
    return bar

In [9]:
# var is assigned to `bar` because we are returning it inside `foo`
var = foo()  

In [10]:
var()

42


But wait a minute, how does var know anything about x? x is defined in foo's scope, not bar's. You would think that x cannot be accessible outside the scope of foo. That's where closures come in.

Closure is a built-in memory of a function that contains all the nonlocal names (in a tuple) the function needs to run!

In [11]:
var.__closure__

(<cell at 0x7f86b3154f70: int object at 0x7f86b5b20610>,)

In [12]:
var.__closure__[0].cell_contents

42

Once you access the closure of a function as a tuple, it will contain elements called cells with the value of a single nonlocal argument. There can be as many cells inside closure as the function needs:

In [13]:
outside = 'global variable'  # not in closure because it is a global variable

def parent():
    x = 100     # x, y, x will be added to closure because they are nonlocal
    y = 'Hello'
    z = {'name': 'Jon',
         'surname': 'smith'}
    
    def child():
        # as we are printing x, y, and z, they get added to the closure
        print(x, y, z)
        value = 42  # not in the closure because value is not nonlocal
    
    return child

# func is now assigned to `child` function
func = parent()

# print each cell in func's closure
for cell in func.__closure__:
    print(cell.cell_contents, '\n')

100 

Hello 

{'name': 'Jon', 'surname': 'smith'} 



In [14]:
outside = 'global variable'  # not in closure because it is a global variable

def parent():
    x = 100     # x, y, x will be added to closure because they are nonlocal
    y = 'Hello'
    z = {'name': 'Jon',
         'surname': 'smith'}
    
    def child():
        # as we are printing x, y, and z, they get added to the closure
        print(x, y)
        value = 42  # not in the closure because value is not nonlocal
    
    return child

# func is now assigned to `child` function
func = parent()

# print each cell in func's closure
for cell in func.__closure__:
    print(cell.cell_contents, '\n')

100 

Hello 



Now, consider this trickier example:

In [15]:
var = 'dummy'

def parent(arg):
    
    def child():
        print(arg)
    
    return child
    
func = parent(var)

In [16]:
func()

dummy


In [17]:
# Delete 'var'
del var

# call func again
func()

dummy


It still prints out ‘dummy’. Why?

You guessed it, it got added to the closure! So, when a value from outer levels of scope gets added to closure, it will stay there unchanged even if we delete the original value!

In [18]:
func.__closure__[0].cell_contents

'dummy'

If we did not delete var and changed its value, the closure would still contain its old value:

In [19]:
var = 'dummy'

def parent(arg):
    
    def child():
        print(arg)
    
    return child

# Call it as is
my_func = parent(var)
my_func()

# Call after changing var
var = 'new_dummy'
my_func()

dummy
dummy


- The closure is an internal memory of a nested function, and it contains all the nonlocal variables stored in a tuple.
- Once a value is stored in a closure, it can be accessed but cannot be overridden if the original value gets deleted or modified
- A nested function is a function defined in another and follows this general pattern:

`>>> def parent(arg):`
    
`...     def child():`

`...        print(arg)`

`...    `

`...    return child`

## Decorators

In [20]:
def add_one(func):

    def wrapper(a):
        return func(a + 1)

    return wrapper

In [27]:
@add_one
def square(a):
    return a ** 2

square(5)

36

## Real-world Examples With Decorators

Create a timer decorator:

In [29]:
import time


def timer(func):
    """
    A decorator to calculate how long a function runs.
    
    Parameters
    ----------
    func: callable
      The function being decorated.
      
    Returns
    -------
    func: callable
      The decorated function.
    """
    def wrapper(*args, **kwargs):
        # Start the timer
        start = time.time()
        # Call the `func`
        result = func(*args, **kwargs)
        # End the timer
        end = time.time()
        
        print(f"{func.__name__} took {round(end - start, 4)} seconds to run!")
        
        return result

    return wrapper

This time, notice how we are using *args and **kwargs. They are used when we don't know the exact number of positional and keyword arguments in the function, which is perfect in this case since we may use timer on any kind of function.

Now, you can use this decorator on any function to determine how long it runs. No repeated code!

In [30]:
@timer
def sleep(n):
    """
    Sleep for n seconds
    
    Parameters
    ----------
    n: int or float
      The number of seconds to wait
    """
    time.sleep(n)


sleep(5)

sleep took 5.0028 seconds to run!


The next very useful decorator would be a caching decorator. Caching decorators are great for computation-heavy functions, which you may call with the same arguments many times. Caching the results of each function call in the closure will let us immediately return the result if the decorated function gets called with known values:

In [31]:
def cache(func):
    """
    A decorator to cache/memoize func's results
    
    Parameters
    ----------
    func: callable
      The function being decorated
    
    Returns
      func: callable
        The decorated function
    """
    # Create a dictionary to store results
    cache = {}  # this will be stored in closure because it is nonlocal
    
    def wrapper(*args, **kwargs):
        # Unpack args and kwargs into a tuple to be used as dict keys
        keys = (tuple(args) + tuple(kwargs.keys()))
        # If not seen before
        if keys not in cache:
            # Store them in cache
            cache[keys] = func(*args, **kwargs)
        # Else return the recorded result
        return cache[keys]
    
    return wrapper

he caching dictionary would look like this:

`cache = {`

`    (arg1, arg2, arg3): func(arg1, arg2, arg3)`

`}`

In [32]:
@timer
@cache
def sleep(n):
    """
    Sleep for n seconds
    
    Parameters
    ----------
    n: int or float
      The number of seconds to wait
    """
    time.sleep(n)

In [33]:
sleep(10)

wrapper took 10.0098 seconds to run!


In [34]:
sleep(10)

wrapper took 0.0 seconds to run!


## Decorators That Take Arguments

So far, our knowledge of decorators is pretty solid. However, the real power of decorators comes when you enable them to take arguments.

Consider this decorator, which checks if the function’s result is of type `str`:

In [35]:
def is_str(func):
    """
    A decorator to check if `func`'s result is a string
    """
    def wrapper(*args, **kwargs):
        # Call func
        result = func(*args, **kwargs)
        return type(result) == str
    return wrapper

In [36]:
@is_str
def foo(arg):
    return arg

In [37]:
foo(4)

False

In [38]:
foo("Python")

True

It would be cool if we had a way to check the function’s return type for any data type

In [40]:
def is_type(dtype):
    """
    Defines a decorator and returns it.
    """
    def decorator(func):
        """
        A decorator to check if func's result is of type `dtype`
        """
        def wrapper(*args, **kwargs):
            result = func(*args, **kwargs)
            return type(result) == dtype
        return wrapper
    return decorator

In [41]:
@is_type(dict)
def foo(arg):
    return arg

foo({1: 'Python'})

True

In [42]:
@is_type(int)
def square(num):
    return num ** 2

square(12)

True

First, let’s just create a simple decorator that calls whatever function passed to it:

In [43]:
def decorator(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result
    return wrapper

In [44]:
def is_type(dtype):
    """
    Defines a decorator and returns it.
    
    Parameters
    ----------
    dtype: type class
      Any data type class such as str, int, float, etc.
    
    Returns
    -------
    func: callable
      A decorator that checks the return type of a function.
    """
    def decorator(func):
        """
        A decorator that checks the return type of `func`.
        """
        def wrapper(*args, **kwargs):
            # Call `func` with given arguments
            result = func(*args, **kwargs)
            # Return True or False depending if types match or not
            return type(result) == dtype
        return wrapper
    return decorator

In [46]:
@is_type(tuple)
def return_tuple(arg):
    return arg

In [47]:
return_tuple((1, 2, 3, 4))

True

## Preserving the Decorated Function’s Metadata

In [48]:
@timer
def sleep(n=5):
    """
    A function to sleep for n seconds.
    
    Parameters
    ----------
    n: int
      Number of seconds to sleep.
    """
    time.sleep(n)

In [50]:
# Call the function for an example
sleep(3)

sleep took 3.0013 seconds to run!


In [51]:
# Checking metadata
# Extracting the doc string
sleep.__doc__

In [52]:
# Get the default arguments
sleep.__defaults__

In [53]:
# Get the name
sleep.__name__

'wrapper'

`sleep` had a long docstring and a default argument that was equal to 5. Where did they go? We got the answer when we called `__name__` and got `wrapper` for the function name.

We can see that we are not actually returning the passed function but returning it inside `wrapper`. Obviously `wrapper` does not have a docstring or any default arguments, which was why we got `None` above.

To solve this problem, Python provides us with a helpful function from `functools` module:

In [54]:
from functools import wraps

In [55]:
def timer(func):
    """A decorator to calculate how long a function runs.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        # Start the timer
        start = time.time()
        # Call the `func`
        result = func(*args, **kwargs)
        # End the timer
        end = time.time()
        
        print(f"{func.__name__} took {round(end - start, 4)} seconds to run!")
        return result
    return wrapper

Using `wraps` on the `wrapper` function lets us keep all the metadata attached to `func`. Notice how we are passing `func` to `wraps` above the function definition.

If we use this modified version of `timer`, we will see that it works as expected:

In [56]:
@timer
def sleep(n=5):
    """
    A function to sleep for n seconds.
    
    Parameters
    ----------
    n: int
      Number of seconds to sleep.
    """
    time.sleep(n)

In [57]:
sleep.__name__

'sleep'

In [60]:
sleep.__doc__

'\n    A function to sleep for n seconds.\n    \n    Parameters\n    ----------\n    n: int\n      Number of seconds to sleep.\n    '